# Text Pre-processing

Functions used in the main code.

In [ ]:
# Text Cleaning

def clean_up(dataf, text, str_punct, remove_stopwd=False, remove_emoji=True):
    clean_dataf = dataf.copy()
    tk = TweetTokenizer()
    stopwd = stopwords.words("english")
    clean_txt = []

    if text == "Tweet":
        text_list = clean_dataf["Tweet"].tolist()
    elif text == "Lyrics":
        text_list = clean_dataf["Lyrics"].tolist()

    for txt in text_list:
        if type(txt) == str:
            txt_free = re.sub(r'\d+', '', txt.lower())  # remove digits
            if text == "Tweet":
                txt_free = re.sub("@[A-Za-z0-9]+", "", txt_free)  # remove mentions "@user"
            txt_free = txt_free.translate(
                str.maketrans(str_punct, ' ' * len(str_punct)))  # remove punctuation and add blank space
            if remove_emoji:
                txt_free = re.sub(r":[^>]+:", "", emoji.demojize(txt_free))  # remove emoji ":emoji_code:"
            if remove_stopwd:
                clean_txt.append(" ".join([wd for wd in tk.tokenize(txt_free) if wd not in stopwd]))  # remove stopwords
            elif not remove_stopwd:
                clean_txt.append(" ".join(tk.tokenize(txt_free)))
        elif type(txt) != str and text == "Tweet":
            clean_txt.append(txt)

    clean_dataf[text] = clean_txt
    return clean_dataf

In [ ]:
# Text Lemmatization, Tokenization and Part of Speech (POS) tagging

def tk_POStag(clean_dataf, lemmatize=True):
    tk = TweetTokenizer()
    tk_dataf = clean_dataf.copy()
    text_list = clean_dataf["Tweet"].tolist()

    if lemmatize:
        lemmtzr = WordNetLemmatizer()
        lemm_text = []
        for txt in text_list:
            lemm_text.append(" ".join([lemmtzr.lemmatize(wd) for wd in txt.split()]))
        tk_dataf["Tweet"] = lemm_text

    tk_dataf["Tokens POS"] = [pos_tag(tk.tokenize(txt)) for txt in
                              tk_dataf["Tweet"]]  # additional column with list of POS tags

    return tk_dataf